这个章节书上有对应的脚本，但是在 开源文件中的ch10里面，这里的 ekf在前面的优化和前面理论部分类似

简单的来说就是假设一个代价函数，用高斯牛顿或者凸优化的方法求极限值

可以将点云图优化的更清楚，降低波动和随机的影响

In [12]:
import numpy as np
from scipy.optimize import least_squares
import time

class BALProblem:
    def __init__(self, filename):
        with open(filename, 'r') as f:
            self.num_cameras, self.num_points, self.num_observations = map(int, f.readline().split())
            self.camera_indices = np.zeros(self.num_observations, dtype=int)
            self.point_indices = np.zeros(self.num_observations, dtype=int)
            self.observations = np.zeros((self.num_observations, 2))
            
            for i in range(self.num_observations):
                self.camera_indices[i], self.point_indices[i], x, y = map(float, f.readline().split())
                self.observations[i] = [x, y]
            
            # 读取剩余的所有数据
            remaining_data = np.fromfile(f, sep=' ', dtype=float)
            
            # 计算相机参数和点的总数
            total_camera_params = self.num_cameras * 9
            total_point_params = self.num_points * 3
            
            if len(remaining_data) != total_camera_params + total_point_params:
                print(f"Warning: Data size mismatch. Expected {total_camera_params + total_point_params}, got {len(remaining_data)}")
            
            # 分割数据为相机参数和点
            self.cameras = remaining_data[:total_camera_params].reshape((self.num_cameras, 9))
            self.points = remaining_data[total_camera_params:].reshape((self.num_points, 3))



    def WriteToPLYFile(self, filename):
        with open(filename, 'w') as f:
            f.write("ply\n")
            f.write("format ascii 1.0\n")
            f.write(f"element vertex {self.num_points}\n")
            f.write("property float x\n")
            f.write("property float y\n")
            f.write("property float z\n")
            f.write("end_header\n")
            for point in self.points:
                f.write(f"{point[0]} {point[1]} {point[2]}\n")

    def Normalize(self):
        # Implement normalization if needed
        pass

    def Perturb(self, rotation_sigma, translation_sigma, point_sigma):
        # Implement perturbation if needed
        pass
import numpy as np

def project(point, camera):
    """
    Project a 3D point using the camera parameters.
    This is a simplified projection model and may need to be adjusted based on your specific camera model.
    """
    # Extract camera parameters
    rotation = camera[:3]
    translation = camera[3:6]
    focal_length = camera[6]
    k1, k2 = camera[7:]

    # Rotate and translate the point
    p = rotation.dot(point) + translation

    # Perspective division
    p_x, p_y = p[:2] / p[2]

    # Apply radial distortion
    r_squared = p_x**2 + p_y**2
    distortion = 1.0 + k1 * r_squared + k2 * r_squared**2

    # Apply focal length and distortion
    x = focal_length * distortion * p_x
    y = focal_length * distortion * p_y

    return np.array([x, y])

def compute_residuals(params, n_cameras, n_points, camera_indices, point_indices, observations):
    """
    Compute residuals.
    params: concatenated camera parameters and 3D point coordinates
    """
    camera_params = params[:n_cameras * 9].reshape((n_cameras, 9))
    points_3d = params[n_cameras * 9:].reshape((n_points, 3))

    residuals = []
    for camera_index, point_index, observation in zip(camera_indices, point_indices, observations):
        camera = camera_params[int(camera_index)]
        point = points_3d[int(point_index)]
        
        projected = project(point, camera)
        residuals.append(observation - projected)

    return np.array(residuals, dtype=np.float64).ravel()


def bundle_adjustment(bal_problem):
    n_cameras = bal_problem.num_cameras
    n_points = bal_problem.num_points
    n_observations = bal_problem.num_observations

    x0 = np.hstack((bal_problem.cameras.ravel(), bal_problem.points.ravel()))
    
    # 确保所有输入都是浮点数
    x0 = x0.astype(np.float64)
    bal_problem.camera_indices = bal_problem.camera_indices.astype(np.int32)
    bal_problem.point_indices = bal_problem.point_indices.astype(np.int32)
    bal_problem.observations = bal_problem.observations.astype(np.float64)

    f0 = compute_residuals(x0, n_cameras, n_points, bal_problem.camera_indices,
                           bal_problem.point_indices, bal_problem.observations)
    
    if not isinstance(f0, np.ndarray):
        raise TypeError(f"compute_residuals returned {type(f0)}, expected numpy.ndarray")
    
    if not np.all(np.isfinite(f0)):
        print("Warning: Initial residuals contain non-finite values. Proceeding with optimization...")

    res = least_squares(compute_residuals, x0, verbose=2, x_scale='jac', ftol=1e-4, method='trf',
                        args=(n_cameras, n_points, bal_problem.camera_indices, bal_problem.point_indices,
                              bal_problem.observations))

    return res.x


def main(filename, params):
    bal_problem = BALProblem(filename)

    print(f"BAL problem file loaded...")
    print(f"BAL problem has {bal_problem.num_cameras} cameras and {bal_problem.num_points} points.")
    print(f"Forming {bal_problem.num_observations} observations.")

    if params.initial_ply:
        bal_problem.WriteToPLYFile(params.initial_ply)

    bal_problem.Normalize()
    bal_problem.Perturb(params.rotation_sigma, params.translation_sigma, params.point_sigma)

    print("Beginning optimization...")
    start_time = time.time()
    x = bundle_adjustment(bal_problem)
    end_time = time.time()

    print(f"Optimization took {end_time - start_time:.3f} seconds.")

    # Update BALProblem with optimized parameters
    n_cameras = bal_problem.num_cameras
    n_points = bal_problem.num_points
    bal_problem.cameras = x[:n_cameras * 9].reshape((n_cameras, 9))
    bal_problem.points = x[n_cameras * 9:].reshape((n_points, 3))

    if params.final_ply:
        bal_problem.WriteToPLYFile(params.final_ply)

if __name__ == "__main__":
    class Args:
        def __init__(self):
            self.input = "/Users/bytedance/Desktop/test/slambook_python/ch10/g2o_custombundle/data/problem-16-22106-pre.txt"  # 替换为你的输入文件路径
            self.initial_ply = ""
            self.final_ply = "result.ply"
            self.rotation_sigma = 0.0
            self.translation_sigma = 0.0
            self.point_sigma = 0.0

    args = Args()
    main(args.input, args)



BAL problem file loaded...
BAL problem has 16 cameras and 22106 points.
Forming 83718 observations.
Beginning optimization...
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         5.3166e+41                                    8.39e+55    


ValueError: Indexing a matrix of 11128131432 elements would incur an in integer overflow in LAPACK.